# Import, install, uninstall necessary libraries¶


In [2]:
import warnings
warnings.filterwarnings('ignore')

!pip uninstall -y ibm-aigov-facts-client --quiet 
!pip install --extra-index-url https://test.pypi.org/simple/ ibm-aigov-facts-client --quiet 
    
!pip install -U ibm-watson-studio-pipelines | tail -n 1
!pip install ibm-aigov-facts-client | tail -n 1
    
from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig

In [3]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from project_lib import Project

#sklearn 
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
import os, types
from sklearn.metrics import f1_score
import numpy as np
import joblib
import datetime
import tempfile
from ibm_watson_studio_pipelines import WSPipelines

#WML
from ibm_watson_machine_learning import APIClient

#AiGovernance
from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig
import sys

# Define object for AIFactSheets, cloudPakforData credentials and for WML

In [3]:
from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig
creds=CloudPakforDataConfig(service_url="https://cpd-cpd-instance.apps.ibmpepsicopoc04.eastus.aroapp.io",
    username="admin",
    password="NXkMMLzehqMX")

In [4]:
api_key = "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6InAwbGRmU084aXRldHdWS1d6ZEQ4ZXY2M2NhTi0yT29IUklSZnhfQWxNLXMifQ.eyJ1aWQiOiIxMDAwMzMwOTk5IiwidXNlcm5hbWUiOiJhZG1pbiIsInJvbGUiOiJBZG1pbiIsInBlcm1pc3Npb25zIjpbImFkbWluaXN0cmF0b3IiLCJjYW5fcHJvdmlzaW9uIiwibWFuYWdlX2NhdGFsb2ciLCJjcmVhdGVfcHJvamVjdCIsImNyZWF0ZV9zcGFjZSIsImF1dGhvcl9nb3Zlcm5hbmNlX2FydGlmYWN0cyIsIm1hbmFnZV9nb3Zlcm5hbmNlX3dvcmtmbG93Iiwidmlld19nb3Zlcm5hbmNlX2FydGlmYWN0cyIsIm1hbmFnZV9jYXRlZ29yaWVzIiwibWFuYWdlX2dsb3NzYXJ5IiwibWFuYWdlX3F1YWxpdHkiLCJtYW5hZ2VfZGlzY292ZXJ5IiwibWFuYWdlX21ldGFkYXRhX2ltcG9ydCIsImFjY2Vzc19jYXRhbG9nIiwidmlld19xdWFsaXR5Il0sImdyb3VwcyI6WzEwMDAwXSwic3ViIjoiYWRtaW4iLCJpc3MiOiJLTk9YU1NPIiwiYXVkIjoiRFNYIiwiaWF0IjoxNjYzMTc1MjI1LCJleHAiOjUyNjMxNzE2MjV9.nzhqVuki0purrgz7viPCMwPVijVXu8ipyYpky09H4R9taeLBPiPHYY74aX2Ox5Flh1htbF2eTxoyFeKn4VHBpCgtjONhnEhpnDNXWfqxOkYKBI6Mz0o1zrzR8shduxYJ9wtMON9rU4iYZxvmG8I5LcsqFtqU6MY-U283YBZzMjH3rbStpTIygAc-KCIuWuaeqxzGOPh7OGRKz8AVbvMGtKfWfZENgQ2789xbQhgXWz7lnR8WCHg7w2NBhMIYeykIvPOMtTzRnT9OveZDmHWjlLnAr-FQ7DBdiAQ-8YEBAZyQreqwDg7Edu1S_e_M8_jdHwbpIVkyAgwLKFYdTawzHA"

In [5]:
WML_CREDENTIALS = {
                   "url": "https://cpd-cpd-instance.apps.ibmpepsicopoc04.eastus.aroapp.io",
                   "username": "admin",
                   "password" : "NXkMMLzehqMX",
                   "instance_id": "wml_local",
                   "version" : "4.5" #If your env is CP4D 4.0 then specify "4.0" instead of "3.5"
                  }

In [6]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.222'

In [7]:
SPACE_ID = "cd06b153-493a-4252-995e-6188f2ce9c00"
PROJECT_ID = '6a919126-4bdc-40b2-b6e4-4cb53709e351'
output_model_file = "{}_clasification_model".format(str(datetime.datetime.now())[:16]).replace(":", "_").replace(" ", "_")
EXPERIMENT_NAME= output_model_file+"_AI_Facts"

wml_client.set.default_project(os.environ["PROJECT_ID"])

'SUCCESS'

In [8]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
print("Software Specification ID: {}".format(software_spec_uid))

model_props = {
        wml_client._models.ConfigurationMetaNames.NAME:"{}".format(output_model_file),
        wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.0",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
        wml_client._models.ConfigurationMetaNames.LABEL_FIELD:"target",
    }

Software Specification ID: 12b83a17-24d8-5082-900f-0ab31fbfd3cb


In [9]:
os.environ["PROJECT_ID"]

'6a919126-4bdc-40b2-b6e4-4cb53709e351'

In [58]:
# api_key = "Ywq3pJq7lLA09jvTmJ13rcXOwJZ2ScX822xt1Lz0"
# from ibm_aigov_facts_client import AIGovFactsClient
# facts_client = AIGovFactsClient(
#                                 experiment_name="MLopsTestsimple"
#                                 ,container_type="project"
#                                 ,container_id=os.environ["PROJECT_ID"]
#                                 ,set_as_current_experiment=True
#                                 ,api_key=api_key)

In [10]:

from ibm_aigov_facts_client import AIGovFactsClient
facts_client = AIGovFactsClient(
                                experiment_name="MLopsTestsimple"
                                ,container_type="project"
                                ,container_id=os.environ["PROJECT_ID"]
                                ,set_as_current_experiment=True
                                ,cloud_pak_for_data_configs=creds)

KeyError: 'api_key'

# Read Model Training and Test Data

In [4]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

nb_data_request = {
    'data_name': """clasification_train_FE.csv""",
    'interaction_properties': {
        #'row_limit': 500,
        'infer_schema': 'true',
        'infer_as_varchar': 'false'
    }
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

data_df_2 = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
data_df_2.head(10)
df_FE = data_df_2

In [5]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

nb_data_request = {
    'data_name': """clasification_test_FE.csv""",
    'interaction_properties': {
        #'row_limit': 500,
        'infer_schema': 'true',
        'infer_as_varchar': 'false'
    }
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

data_df_3 = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
data_df_3.head(10)
df_test_FE= data_df_3

In [6]:
test_x = df_test_FE.drop(["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1","Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2","Attrition_Flag"], axis=1)
test_y = df_test_FE["Attrition_Flag"]
df_test_FE.head()

,Customer_Age,Gender,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,...,$40K - $60K,$60K - $80K,$80K - $120K,Less than $40K,Divorced,Married,Single,Blue,Gold,Silver
0,39,True,4,36,5,3,4,1623.0,729,894.0,...,False,False,False,True,False,False,True,True,False,False
1,33,False,3,24,6,3,3,5156.0,1427,3729.0,...,True,False,False,False,False,False,True,True,False,False
2,48,False,4,30,4,2,2,3307.0,2234,1073.0,...,False,True,False,False,False,False,True,True,False,False
3,47,True,2,34,4,2,4,2730.0,1970,760.0,...,False,False,False,True,False,True,False,True,False,False
4,57,False,1,48,5,1,3,2544.0,1927,617.0,...,True,False,False,False,False,True,False,True,False,False


In [7]:
train_x = df_FE.drop(["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1","Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2","Attrition_Flag"], axis=1)
train_y = df_FE["Attrition_Flag"]


# Model Develpment

In [8]:
rf_pipe = Pipeline(steps =[ ('scale',StandardScaler()), ("RF",RandomForestClassifier(random_state=42)) ])
ada_pipe = Pipeline(steps =[ ('scale',StandardScaler()), ("ADA",AdaBoostClassifier(random_state=42,learning_rate=0.7)) ])
svm_pipe = Pipeline(steps =[ ('scale',StandardScaler()), ("SVC",SVC(kernel='rbf',C=2,probability=True)) ])

In [10]:
models = {"RF": None, "ADA": None, "SVC": None} 
models_run_id = {"RF": "", "ADA": "", "SVC": ""} 


models["RF"] = rf_pipe.fit(train_x,train_y)
#print("printing run ID:",facts_client.runs.get_current_run_id())
#models_run_id["RF"] = facts_client.runs.get_current_run_id()
rf_prediction = rf_pipe.predict(test_x)

models["ADA"] =  ada_pipe.fit(train_x,train_y)
#print("printing run ID:",facts_client.runs.get_current_run_id())
#models_run_id["ADA"] = facts_client.runs.get_current_run_id()
ada_prediction = ada_pipe.predict(test_x)

models["SVC"] = svm_pipe.fit(train_x,train_y)
#print("printing run ID:",facts_client.runs.get_current_run_id())
#models_run_id["SVC"] = facts_client.runs.get_current_run_id()
svm_prediction = svm_pipe.predict(test_x)

In [15]:
import pickle
from project_lib import Project
project = Project()
pc = project.project_context

In [16]:
project.save_data(data=pickle.dumps(rf_pipe),file_name='rf_pipe.pkl',overwrite=True)

{'file_name': 'rf_pipe.pkl',
 'message': 'File saved to project storage.',
 'asset_id': '930fc2f0-3cbb-4a47-a5a0-fe173688a31e'}

In [17]:
project.save_data(data=pickle.dumps(ada_pipe),file_name='ada_pipe.pkl',overwrite=True)

{'file_name': 'ada_pipe.pkl',
 'message': 'File saved to project storage.',
 'asset_id': '4757a1c1-761d-4553-8a81-c1036dd78f13'}

In [18]:
project.save_data(data=pickle.dumps(svm_pipe),file_name='svm_pipe.pkl',overwrite=True)

{'file_name': 'svm_pipe.pkl',
 'message': 'File saved to project storage.',
 'asset_id': 'ba72e6e1-2046-49ee-9ada-09dd3076b65e'}

In [ ]:
models_run_id

In [ ]:
import numpy as np


rf_score = np.round(f1_score(rf_prediction,test_y),2)
ada_score = np.round(f1_score(ada_prediction,test_y),2)
svm_score = np.round(f1_score(svm_prediction,test_y),2)

scores = [(rf_score, "RF"), (ada_score, "ADA"), (svm_score, "SVC")]
print(scores)
print(models_run_id)
print(models)

best_model = sorted(scores, key=lambda x: x[0])[-1][-1]
best_mode_run_id = models_run_id[best_model]
best_model = models[best_model]

print("best model:", best_model)
print(best_mode_run_id)

# best_model

       
# joblib.dump(best_model, output_model_file)
# project.save_data(output_file, df_train.to_csv(index=False, header=True),overwrite=True)
print(output_model_file)
# with tempfile.TemporaryFile() as fp:
#     joblib.dump(best_model, fp)
#     fp.seek(0)
#     project.save_data(output_model_file, fp.read(),overwrite=True)

# Export AIFactSheets

In [ ]:
facts_client.export_facts.export_payload(best_mode_run_id)

In [ ]:
facts_client.export_facts.prepare_model_meta(wml_client=wml_client,meta_props=model_props)

In [ ]:
print("Storing model .....")

published_model_details = wml_client.repository.store_model(model=rf_pipe, meta_props=model_props, training_data=df_FE.drop(["Attrition_Flag"], axis=1), training_target=df_FE.Attrition_Flag)
model_uid = wml_client.repository.get_model_id(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))